In [5]:
import pandas as pd
import mne
import datetime
import json
import re

In [6]:
def read_eeg(root):
    
    vhdr_fname = root + trial_day.split("/")[0] + "_" + trial_num.split("/")[0] + "_EEG01.vhdr"
    
    # define list of indicies for non-eeg channels
    misc_list = []
    for i in range(18):
        misc_list.append(i+64)

    # read raw data
    raw = mne.io.read_raw_brainvision(vhdr_fname, misc=misc_list, preload=True,
        verbose=False)
    raw.info['line_freq'] = 500.

    # Set montage
    montage = mne.channels.make_standard_montage('easycap-M1')
    raw.set_montage(montage, verbose=False)

    # Set common average reference
    raw.set_eeg_reference('average', projection=False, verbose=False)

    # create pandas dataframe with eeg data
    eeg_data = pd.DataFrame(raw.get_data().transpose(), columns=raw.ch_names)

    # Temporarily drop T7 while it's broken
    #eeg_data = eeg_data.drop(columns=["T7"])

    return eeg_data


def eeg_times(root):
    
    vhdr_path = root + trial_day.split("/")[0] + "_" + trial_num.split("/")[0] + "_EEG01.vhdr"


    with open(vhdr_path, "r+") as file:

       #read the file contents
       file_contents = file.read()
       text_pattern = re.compile(re.escape("%"))
       file_contents = text_pattern.sub("", file_contents)
       file.seek(0)
       file.truncate()
       file.write(file_contents)

    with open(vhdr_path, "r") as r:
        vhdr_lines = r.readlines()
    
    eeg_start = ""

    for i,j in enumerate(vhdr_lines):
        if 'Recording Start Time' in j:
            eeg_start = vhdr_lines[i+1]
    eeg_start = eeg_start.splitlines()[0] #.split(".")[0]

    eeg_start_dt = datetime.datetime.strptime(eeg_start,'%Y-%m-%d %H:%M:%S.%f')

    return eeg_start_dt

def tobii_times(root):

    with open(root + 'segment.json','r') as f:
        seg_data = json.load(f)

    seg_start = seg_data['seg_t_start']
    seg_end = seg_data['seg_t_stop']

    seg_start = seg_start.replace("T"," ").replace("+",".")
    seg_start_dt = datetime.datetime.strptime(seg_start, '%Y-%m-%d %H:%M:%S.%f')

    seg_end = seg_end.replace("T"," ").replace("+",".")
    seg_end_dt = datetime.datetime.strptime(seg_end, '%Y-%m-%d %H:%M:%S.%f')
    
    return seg_start_dt, seg_end_dt

def EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df):
    fs = 500    
    skip_eeg_data = round((tobii_start - eeg_start).total_seconds()*fs) # 500 is the frequency

    total_event_duration = (tobii_end - tobii_start).total_seconds()
    event_eeg_duration = total_event_duration*fs

    eeg_data = eeg_df.iloc[skip_eeg_data:int(skip_eeg_data + event_eeg_duration)].reset_index().drop('index', axis=1)
    eeg_data['dateTime'] = pd.date_range(start=tobii_start, end=tobii_end, periods=len(eeg_data))
    eeg_data.index = eeg_data['dateTime']
    return eeg_data

In [7]:
%%time
# trials = [
#           ["2022_01_14/","T06_U002/"],
#           ["2022_01_27/","T03_U003/"], 
#           ["2022_02_09/","T03_U004/"],
#          ]
trials = [
    ["2021_05_26/", "T03_U000/"],
    ["2021_05_26/", "T05_U000/"],
    ["2021_06_09/", "T03_U000/"],
    ["2021_06_09/", "T04_U000/"],
    ["2021_06_10/", "T03_U000/"],
    ["2021_06_10/", "T04_U000/"],
]


for i in trials:
    trial_day, trial_num, root = i[0], i[1], i[0] + i[1]
    # print(trial_day, trial_num, root)
    eeg_start = eeg_times(root)
    tobii_start, tobii_end = tobii_times(root)
    eeg_df_full = read_eeg(root)
    eeg_df = EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df_full)
    
eeg_df

Wall time: 6.3 s


,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,...,GSR,Temp.,AUX 1,AUX 2,ACC77,ACC78,ACC79,Packet Counter,TRIGGER,dateTime
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.000000000,0.001702,0.004508,-0.008591,-0.004093,-0.000593,0.002736,0.006220,0.004985,0.003791,0.000327,...,11666.615486,29.7500,-0.041909,-0.070300,-4073.660851,10860.770226,-3843.471050,19853.0,0.0,2021-06-10 11:21:49.000000000
2021-06-10 11:21:49.002000012,0.001698,0.004494,-0.008588,-0.004090,-0.000590,0.002733,0.006221,0.004986,0.003783,0.000322,...,11666.615486,29.7500,-0.041881,-0.070267,-4073.660851,10825.892448,-3822.544575,19854.0,0.0,2021-06-10 11:21:49.002000012
2021-06-10 11:21:49.004000024,0.001694,0.004482,-0.008588,-0.004090,-0.000589,0.002735,0.006220,0.004988,0.003780,0.000324,...,11666.615486,29.7500,-0.041881,-0.070276,-4164.341450,10832.868576,-3822.544575,19855.0,0.0,2021-06-10 11:21:49.004000024
2021-06-10 11:21:49.006000036,0.001691,0.004489,-0.008589,-0.004097,-0.000592,0.002738,0.006217,0.004988,0.003786,0.000327,...,11666.615486,29.7500,-0.041924,-0.070262,-4171.317101,10867.745399,-3829.519987,19856.0,0.0,2021-06-10 11:21:49.006000036
2021-06-10 11:21:49.008000049,0.001693,0.004494,-0.008589,-0.004100,-0.000593,0.002740,0.006213,0.004986,0.003780,0.000325,...,11666.615486,29.7500,-0.041928,-0.070210,-4164.341450,10902.623177,-3836.495638,19857.0,0.0,2021-06-10 11:21:49.008000049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-10 11:27:13.991999950,0.000386,0.004217,-0.007306,0.000281,-0.002214,0.000334,0.004104,0.004170,0.003691,0.000158,...,10829.502106,29.4375,-0.044713,-0.156250,-2001.953125,11621.093750,3027.343750,18508.0,0.0,2021-06-10 11:27:13.991999950
2021-06-10 11:27:13.993999963,0.000377,0.004208,-0.007313,0.000283,-0.002217,0.000339,0.004107,0.004177,0.003719,0.000176,...,10829.502106,29.4375,-0.044737,-0.156250,-1974.051356,11851.283073,3229.631662,18509.0,0.0,2021-06-10 11:27:13.993999963
2021-06-10 11:27:13.995999975,0.000368,0.004197,-0.007321,0.000279,-0.002223,0.000338,0.004110,0.004184,0.003739,0.000206,...,10829.502106,29.4375,-0.044665,-0.156250,-2022.879362,12018.693924,3306.361675,18510.0,0.0,2021-06-10 11:27:13.995999975


In [8]:
import mne
import numpy as np
import pandas as pd

# import functions
from scipy import signal
# from biometricVar_per_epoch import *

def getEpochbm_dict(eeg_data, epoch_dict):

    # ignoring T7 
    df = eeg_data.iloc[:, 0:64]     # create dataframe for 64 signals
    fs = 500                        # Sampling rate of 500 Hz
    epochbm_dict = dict()           # final dictionary
    
    # Read dictionary with non-eeg data
    # non_eeg_dict = biometricVar_per_epoch(epoch_dict, eeg_data)
    
    # Define EEG frequency bands
    bands = {'Delta': (1, 3),
             'Theta': (4, 7),
             'Alpha': (8, 12),
             'Beta': (13, 25),
             'Gamma': (26, 45)}

    # Outer loop iterates through epoch dictionary and creates a dataframe for
    # each epoch based on its time index range and stores it along with the index
    # ranges in the final dictionary
    for e in epoch_dict:
        start_idx = epoch_dict[e][0]    # beginning time index range
        end_idx = epoch_dict[e][1]      # ending time index range

        eeg_list = []                   # list of band dictionaries
        idx_name = []                   # name of electrodes

        # slice dataframe according to epoch edges
        epoch_df = df[start_idx:end_idx]

        # inner loop goes through each column of the sliced dataframe applies the
        # welch function and stores the dictionaries in a list
        for i in epoch_df:
            idx_name.append(i)          # store name of electrode
            eeg_bands = dict()          # dictionary holding EEG bands as keys and freq as value

            # Take one electrode at a time from dataframe
            data_col = epoch_df.loc[:, i]
            eeg_signal = data_col.values

            # welch function applied to each electrode using sampling rate
            freq_arr, psd_arr = signal.welch(eeg_signal, fs)

            # find freq bands of all electrodes for given epoch and store in dictionary
            for b in bands:
                # Find frequency match with EEG bands
                freq_ix = np.where((freq_arr >= bands[b][0]) & (freq_arr <= bands[b][1]))

                # Calculate the mean of power spectrum value
                eeg_bands[b] = np.mean(psd_arr[freq_ix])

            # add band dictionary to list of dictionaries
            eeg_list.append(eeg_bands)

        # create new dataframe for freq bands of each electrode for given epoch
        bands_df = pd.DataFrame(eeg_list, columns=['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma'], index=idx_name)

        # add time index ranges and both EEG and non-EEG biometric dataframes to final dictionary in each epoch
        epochbm_dict[e] = [[start_idx, end_idx], bands_df]
        # epochbm_dict[e] = [[start_idx, end_idx], bands_df, non_eeg_dict[e][1]]

    return epochbm_dict

idx = [i for i in range(len(eeg_df))]
window = list(zip(idx[:-5*500:500],idx[2500::500]))

window_dict = {}
for i,j in enumerate(window):
    window_dict[i] = j

In [9]:
time_index = eeg_df.index[:-5*500:500]
time_index

DatetimeIndex([          '2021-06-10 11:21:49',
               '2021-06-10 11:21:50.000006153',
               '2021-06-10 11:21:51.000012307',
               '2021-06-10 11:21:52.000018461',
               '2021-06-10 11:21:53.000024615',
               '2021-06-10 11:21:54.000030769',
               '2021-06-10 11:21:55.000036923',
               '2021-06-10 11:21:56.000043077',
               '2021-06-10 11:21:57.000049231',
               '2021-06-10 11:21:58.000055384',
               ...
               '2021-06-10 11:26:59.001907704',
               '2021-06-10 11:27:00.001913857',
               '2021-06-10 11:27:01.001920011',
               '2021-06-10 11:27:02.001926165',
               '2021-06-10 11:27:03.001932319',
               '2021-06-10 11:27:04.001938473',
               '2021-06-10 11:27:05.001944627',
               '2021-06-10 11:27:06.001950781',
               '2021-06-10 11:27:07.001956935',
               '2021-06-10 11:27:08.001963089'],
              dtype=

In [10]:
%%time
big_dict = getEpochbm_dict(eeg_df, window_dict)

Wall time: 8.02 s


In [11]:
%%time
list_of_psds = []
for i,j in big_dict.items():
    list_of_psds.append(j[1].melt()['value'].values)

Wall time: 338 ms


In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [13]:
psds_df = pd.DataFrame(list_of_psds)
# scaler = MinMaxScaler()
scaler = StandardScaler()
psds_df = pd.DataFrame(scaler.fit_transform(psds_df))

In [14]:
psds_df.index = time_index
psds_df

,0,1,2,3,4,5,6,7,8,9,...,310,311,312,313,314,315,316,317,318,319
dateTime,,,,,,,,,,,,,,,,,,,,,
2021-06-10 11:21:49.000000000,-0.246923,-0.488402,4.203363,1.684167,1.747854,1.439680,0.973149,1.641176,-0.563372,-0.642587,...,-0.081074,-1.997291,0.882875,-0.932781,1.074913,1.006320,-0.488533,-1.589389,-0.633187,-1.257833
2021-06-10 11:21:50.000006153,-0.552720,-0.841779,3.820827,1.206165,1.424943,1.117453,0.332017,0.886717,-0.643979,-0.498108,...,0.403326,-1.970247,1.058936,-0.719382,1.444101,2.083434,-0.959646,-1.079837,-0.123607,-0.850877
2021-06-10 11:21:51.000012307,-1.044881,-1.352221,3.471130,0.844111,1.067801,0.895357,-0.021664,0.429636,-0.781440,-0.665250,...,0.636723,-1.918285,1.146909,-0.490226,1.892664,2.560983,-0.909929,-1.060818,0.220585,-0.796098
2021-06-10 11:21:52.000018461,-1.457260,-1.511601,3.738756,1.881970,1.790787,1.671670,0.758621,0.188909,-0.936000,-0.463734,...,0.911202,-1.361097,1.260921,-0.237321,2.229807,3.153182,-0.705215,-1.094158,0.769528,-0.588802
2021-06-10 11:21:53.000024615,-1.678298,-1.638750,-0.231892,0.114523,-0.137143,0.009806,-0.707205,-1.392655,-1.244657,-0.702031,...,0.242123,-1.026599,0.690467,-0.087522,1.313773,1.784112,-0.423898,-0.985645,0.691600,-0.657697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-10 11:27:04.001938473,-0.306734,0.246687,-0.398441,-0.483566,0.186104,0.986930,-0.114351,1.454480,-0.286642,-0.243885,...,-0.789135,-0.693258,-0.804562,-0.620163,-0.674557,-0.799375,-0.116387,-0.462906,-0.511241,-0.902983
2021-06-10 11:27:05.001944627,-0.447475,0.083870,0.016492,-0.294993,0.181091,0.786377,-0.168017,0.916855,-0.504803,-0.523998,...,-0.795555,-1.039793,-0.806346,-1.306710,-0.668799,-0.801771,-0.252713,-1.428432,-0.538050,-1.279486
2021-06-10 11:27:06.001950781,-0.828821,-0.480144,1.601746,0.262893,1.266929,1.651578,-0.572053,0.389681,-0.765641,-0.827745,...,-0.776585,-0.979795,-0.792840,-1.581587,-0.721266,-1.021637,-0.553677,-1.333030,-0.818085,-1.408232


In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# trials = [
#           ["2022_01_14/","T06_U002/"],
#           ["2022_01_27/","T03_U003/"], 
#           ["2022_02_09/","T03_U004/"],
#          ]

trials = [
    ["2021_05_26/", "T03_U000/"],
    ["2021_05_26/", "T05_U000/"],
    ["2021_06_09/", "T03_U000/"],
    ["2021_06_09/", "T04_U000/"],
    ["2021_06_10/", "T03_U000/"],
    ["2021_06_10/", "T04_U000/"],
]

full_psds = pd.DataFrame()
for trial in trials:
    trial_day, trial_num, root = trial[0], trial[1], trial[0] + trial[1]
    # print(trial_day, trial_num, root)
    eeg_start = eeg_times(root)
    tobii_start, tobii_end = tobii_times(root)
    eeg_df_full = read_eeg(root)
    eeg_df = EEG_start_sync(tobii_start, tobii_end, eeg_start, eeg_df_full)
    idx = [k for k in range(len(eeg_df))]
    window = list(zip(idx[:-5*500:500],idx[2500::500]))

    window_dict = {}
    for i,j in enumerate(window):
        window_dict[i] = j
    time_index = eeg_df.index[:-5*500:500]
    print(len(time_index), len(eeg_df))
    
    big_dict = getEpochbm_dict(eeg_df, window_dict)
    list_of_psds = []
    for key, value in big_dict.items():
        list_of_psds.append(value[1].melt()['value'].values)


    psds_df = pd.DataFrame(list_of_psds)
    # scaler = MinMaxScaler()
    scaler = StandardScaler()
    psds_df = pd.DataFrame(scaler.fit_transform(psds_df))
    print(len(psds_df))
    psds_df.index = time_index
    
    full_psds = pd.concat([full_psds, psds_df], axis=0)
    print(f"trial {trial} done")
    

1074 539500
1074
trial ['2021_05_26/', 'T03_U000/'] done
386 195500
386
trial ['2021_05_26/', 'T05_U000/'] done
864 434500
864
trial ['2021_06_09/', 'T03_U000/'] done
1419 712000
1419
trial ['2021_06_09/', 'T04_U000/'] done
823 414000
823
trial ['2021_06_10/', 'T03_U000/'] done
320 162500
320
trial ['2021_06_10/', 'T04_U000/'] done


In [16]:
bands = {'delta': (1, 3),
         'theta': (4, 7),
         'alpha': (8, 12),
         'beta': (13, 25),
         'gamma': (26, 45)}

ch_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7','F8','T7', 'T8',
        'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6',
        'CP5', 'CP6', 'FT9', 'FT10', 'FCz', 'AFz', 'F1', 'F2', 'C1', 'C2', 'P1', 'P2',
        'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6',
        'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'CPz', 'POz', 'TP10']

psd_cols = [j+"-"+i+' ($\mathrm{V^2}$/Hz)' for i in bands.keys() for j in ch_names]

In [17]:
pd.set_option('display.max_columns', None)
full_psds.columns = psd_cols
full_psds

,Fp1-delta ($\mathrm{V^2}$/Hz),Fp2-delta ($\mathrm{V^2}$/Hz),F3-delta ($\mathrm{V^2}$/Hz),F4-delta ($\mathrm{V^2}$/Hz),C3-delta ($\mathrm{V^2}$/Hz),C4-delta ($\mathrm{V^2}$/Hz),P3-delta ($\mathrm{V^2}$/Hz),P4-delta ($\mathrm{V^2}$/Hz),O1-delta ($\mathrm{V^2}$/Hz),O2-delta ($\mathrm{V^2}$/Hz),F7-delta ($\mathrm{V^2}$/Hz),F8-delta ($\mathrm{V^2}$/Hz),T7-delta ($\mathrm{V^2}$/Hz),T8-delta ($\mathrm{V^2}$/Hz),P7-delta ($\mathrm{V^2}$/Hz),P8-delta ($\mathrm{V^2}$/Hz),Fz-delta ($\mathrm{V^2}$/Hz),Cz-delta ($\mathrm{V^2}$/Hz),Pz-delta ($\mathrm{V^2}$/Hz),Oz-delta ($\mathrm{V^2}$/Hz),FC1-delta ($\mathrm{V^2}$/Hz),FC2-delta ($\mathrm{V^2}$/Hz),CP1-delta ($\mathrm{V^2}$/Hz),CP2-delta ($\mathrm{V^2}$/Hz),FC5-delta ($\mathrm{V^2}$/Hz),FC6-delta ($\mathrm{V^2}$/Hz),CP5-delta ($\mathrm{V^2}$/Hz),CP6-delta ($\mathrm{V^2}$/Hz),FT9-delta ($\mathrm{V^2}$/Hz),FT10-delta ($\mathrm{V^2}$/Hz),FCz-delta ($\mathrm{V^2}$/Hz),AFz-delta ($\mathrm{V^2}$/Hz),F1-delta ($\mathrm{V^2}$/Hz),F2-delta ($\mathrm{V^2}$/Hz),C1-delta ($\mathrm{V^2}$/Hz),C2-delta ($\mathrm{V^2}$/Hz),P1-delta ($\mathrm{V^2}$/Hz),P2-delta ($\mathrm{V^2}$/Hz),AF3-delta ($\mathrm{V^2}$/Hz),AF4-delta ($\mathrm{V^2}$/Hz),FC3-delta ($\mathrm{V^2}$/Hz),FC4-delta ($\mathrm{V^2}$/Hz),CP3-delta ($\mathrm{V^2}$/Hz),CP4-delta ($\mathrm{V^2}$/Hz),PO3-delta ($\mathrm{V^2}$/Hz),PO4-delta ($\mathrm{V^2}$/Hz),F5-delta ($\mathrm{V^2}$/Hz),F6-delta ($\mathrm{V^2}$/Hz),C5-delta ($\mathrm{V^2}$/Hz),C6-delta ($\mathrm{V^2}$/Hz),P5-delta ($\mathrm{V^2}$/Hz),P6-delta ($\mathrm{V^2}$/Hz),AF7-delta ($\mathrm{V^2}$/Hz),AF8-delta ($\mathrm{V^2}$/Hz),FT7-delta ($\mathrm{V^2}$/Hz),FT8-delta ($\mathrm{V^2}$/Hz),TP7-delta ($\mathrm{V^2}$/Hz),TP8-delta ($\mathrm{V^2}$/Hz),PO7-delta ($\mathrm{V^2}$/Hz),PO8-delta ($\mathrm{V^2}$/Hz),Fpz-delta ($\mathrm{V^2}$/Hz),CPz-delta ($\mathrm{V^2}$/Hz),POz-delta ($\mathrm{V^2}$/Hz),TP10-delta ($\mathrm{V^2}$/Hz),Fp1-theta ($\mathrm{V^2}$/Hz),Fp2-theta ($\mathrm{V^2}$/Hz),F3-theta ($\mathrm{V^2}$/Hz),F4-theta ($\mathrm{V^2}$/Hz),C3-theta ($\mathrm{V^2}$/Hz),C4-theta ($\mathrm{V^2}$/Hz),P3-theta ($\mathrm{V^2}$/Hz),P4-theta ($\mathrm{V^2}$/Hz),O1-theta ($\mathrm{V^2}$/Hz),O2-theta ($\mathrm{V^2}$/Hz),F7-theta ($\mathrm{V^2}$/Hz),F8-theta ($\mathrm{V^2}$/Hz),T7-theta ($\mathrm{V^2}$/Hz),T8-theta ($\mathrm{V^2}$/Hz),P7-theta ($\mathrm{V^2}$/Hz),P8-theta ($\mathrm{V^2}$/Hz),Fz-theta ($\mathrm{V^2}$/Hz),Cz-theta ($\mathrm{V^2}$/Hz),Pz-theta ($\mathrm{V^2}$/Hz),Oz-theta ($\mathrm{V^2}$/Hz),FC1-theta ($\mathrm{V^2}$/Hz),FC2-theta ($\mathrm{V^2}$/Hz),CP1-theta ($\mathrm{V^2}$/Hz),CP2-theta ($\mathrm{V^2}$/Hz),FC5-theta ($\mathrm{V^2}$/Hz),FC6-theta ($\mathrm{V^2}$/Hz),CP5-theta ($\mathrm{V^2}$/Hz),CP6-theta ($\mathrm{V^2}$/Hz),FT9-theta ($\mathrm{V^2}$/Hz),FT10-theta ($\mathrm{V^2}$/Hz),FCz-theta ($\mathrm{V^2}$/Hz),AFz-theta ($\mathrm{V^2}$/Hz),F1-theta ($\mathrm{V^2}$/Hz),F2-theta ($\mathrm{V^2}$/Hz),C1-theta ($\mathrm{V^2}$/Hz),C2-theta ($\mathrm{V^2}$/Hz),P1-theta ($\mathrm{V^2}$/Hz),P2-theta ($\mathrm{V^2}$/Hz),AF3-theta ($\mathrm{V^2}$/Hz),AF4-theta ($\mathrm{V^2}$/Hz),FC3-theta ($\mathrm{V^2}$/Hz),FC4-theta ($\mathrm{V^2}$/Hz),CP3-theta ($\mathrm{V^2}$/Hz),CP4-theta ($\mathrm{V^2}$/Hz),PO3-theta ($\mathrm{V^2}$/Hz),PO4-theta ($\mathrm{V^2}$/Hz),F5-theta ($\mathrm{V^2}$/Hz),F6-theta ($\mathrm{V^2}$/Hz),C5-theta ($\mathrm{V^2}$/Hz),C6-theta ($\mathrm{V^2}$/Hz),P5-theta ($\mathrm{V^2}$/Hz),P6-theta ($\mathrm{V^2}$/Hz),AF7-theta ($\mathrm{V^2}$/Hz),AF8-theta ($\mathrm{V^2}$/Hz),FT7-theta ($\mathrm{V^2}$/Hz),FT8-theta ($\mathrm{V^2}$/Hz),TP7-theta ($\mathrm{V^2}$/Hz),TP8-theta ($\mathrm{V^2}$/Hz),PO7-theta ($\mathrm{V^2}$/Hz),PO8-theta ($\mathrm{V^2}$/Hz),Fpz-theta ($\mathrm{V^2}$/Hz),CPz-theta ($\mathrm{V^2}$/Hz),POz-theta ($\mathrm{V^2}$/Hz),TP10-theta ($\mathrm{V^2}$/Hz),Fp1-alpha ($\mathrm{V^2}$/Hz),Fp2-alpha ($\mathrm{V^2}$/Hz),F3-alpha ($\mathrm{V^2}$/Hz),F4-alpha ($\mathrm{V^2}$/Hz),C3-alpha ($\mathrm{V^2}$/Hz),C4-alpha ($\mathrm{V^2}$/Hz),P3-alpha ($\mathrm{V^2}$/Hz),P4-alpha (

In [18]:
full_psds.to_pickle("full_psds_for_2021.pkl")